In [3]:
import pandas as pd
import random

# Generate a synthetic music dataset with 1000 songs
num_songs = 1000

# Sample data for genres, artists, and song titles
genres = ['Pop', 'Rock', 'Hip-Hop', 'Jazz', 'Electronic', 'Country', 'R&B', 'Classical']
artists = ['Artist A', 'Artist B', 'Artist C', 'Artist D', 'Artist E','Artist F']
song_titles = [f'Song {i}' for i in range(1, num_songs + 1)]

# Generate random data for attributes
data = {
    'song_title': random.choices(song_titles, k=num_songs),
    'artist': random.choices(artists, k=num_songs),
    'genre': random.choices(genres, k=num_songs),
    'tempo': [random.uniform(60, 200) for _ in range(num_songs)],
    'duration_seconds': [random.randint(120, 360) for _ in range(num_songs)],
    'release_year': [random.randint(2000, 2023) for _ in range(num_songs)],
    'popularity': [random.randint(1, 100) for _ in range(num_songs)],
}

# Create a DataFrame
music_data = pd.DataFrame(data)

# Save the dataset to a CSV file
music_data.to_csv('music_data.csv', index=False)


In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Load the music dataset
music_data = pd.read_csv('music_data.csv')
music_data

,song_title,artist,genre,tempo,duration_seconds,release_year,popularity
0,Song 954,Artist E,Classical,93.984383,204,2017,45
1,Song 171,Artist A,Hip-Hop,187.461950,356,2022,97
2,Song 670,Artist F,Classical,67.189732,247,2013,41
3,Song 888,Artist A,Electronic,123.620369,222,2000,39
4,Song 351,Artist E,R&B,145.926099,189,2020,66
...,...,...,...,...,...,...,...
995,Song 475,Artist E,Pop,126.882867,314,2017,92
996,Song 371,Artist F,Electronic,193.263090,296,2014,21
997,Song 863,Artist B,Country,160.095927,161,2010,44
998,Song 491,Artist F,R&B,85.007818,348,2015,27


In [7]:
# TF-IDF Vectorization for content-based filtering (genres)
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix_genre = tfidf_vectorizer.fit_transform(music_data['genre'])

# Compute the cosine similarity matrix (genres)
cosine_sim_genre = linear_kernel(tfidf_matrix_genre, tfidf_matrix_genre)

# Function to recommend music based on song attributes
def recommend_music(title, cosine_sim_genre=cosine_sim_genre, num_recommendations=10):
    # Check if the title exists in the dataset
    if title not in music_data['song_title'].values:
        return f'Song title "{title}" not found in the dataset.'

    # Get the index of the selected song
    idx = music_data.index[music_data['song_title'] == title].tolist()[0]

    # Calculate similarity scores based on genre
    sim_scores_genre = list(enumerate(cosine_sim_genre[idx]))
    sim_scores_genre = sorted(sim_scores_genre, key=lambda x: x[1], reverse=True)
    sim_scores_genre = sim_scores_genre[1:]  # Exclude the input song itself

    # Get indices of top similar songs based on genre
    genre_indices = [i[0] for i in sim_scores_genre[:num_recommendations]]

    # Get recommendations based on popularity, tempo, and duration
    recommendations = music_data.iloc[genre_indices].sort_values(
        by=['popularity', 'tempo', 'duration_seconds'],
        ascending=[False, True, True]
    )

    return recommendations[['song_title', 'artist', 'genre', 'tempo', 'duration_seconds', 'popularity']]

# User interface
while True:
    print("\nMusic Recommendation System")
    print("1. Recommend Music")
    print("2. Exit")
    choice = input("Enter your choice (1/2): ")

    if choice == '1':
        song_title = input("Enter a song title: ")
        recommendations = recommend_music(song_title)
        if isinstance(recommendations, str):
            print(recommendations)
        else:
            print("\nRecommended Songs:")
            print(recommendations.to_string(index=False))
    elif choice == '2':
        print("Exiting the recommendation system. Goodbye!")
        break
    else:
        print("Invalid choice. Please select 1 or 2.")



Music Recommendation System
1. Recommend Music
2. Exit
Enter your choice (1/2): 1
Enter a song title: Song 670

Recommended Songs:
song_title   artist     genre      tempo  duration_seconds  popularity
  Song 266 Artist A Classical 146.046988               130          93
  Song 185 Artist C Classical 191.170741               153          91
  Song 196 Artist B Classical 161.057239               138          90
  Song 246 Artist A Classical 164.075464               296          82
  Song 341 Artist E Classical  72.214210               316          80
  Song 767 Artist A Classical 164.481590               162          80
  Song 261 Artist C Classical 115.343124               263          66
  Song 160 Artist F Classical 124.424403               172          44
  Song 670 Artist F Classical  67.189732               247          41
  Song 737 Artist F Classical 188.056610               323           6

Music Recommendation System
1. Recommend Music
2. Exit
Enter your choice (1/2): Song 1